# Ali Kamal
# i191865@nu.edu.pk

In [1]:
import spacy
import re
import random
import unicodedata
import numpy as np
from itertools import groupby

In [2]:
#Initializing Spacy
nlp = spacy.load("en_core_web_sm")

# Utility Functions

In [3]:
"""
Gets poetry text file as input and returns a 
stripped list containing the text
"""
def read(f):
    with open(f) as file:
        #lines = ["[START] {0} [END]".format(line.rstrip()) for line in file.readlines() if line.strip()]
        lines = [(line.rstrip()) for line in file.readlines() if line.strip()]
    return [ele for ele in lines if all(ch not in ele for ch in '?')]


"""
Uses Regex to convert a string to upper case
"""
upper_case = re.compile(r'((?<=[\.\?!]\s)(\w+)|(^\w+))')
def capital(string):
    return(string.group().capitalize())


"""
Gets a list of text as input, and returns
ngram (bigram,trigram,4-gram etc) depending
on input parameters
"""
def get_ngrams(doc,n=2):
    if not isinstance(n, int):
        raise Exception(f"Ngram argument {n=} must be an integer type")
    if (n<1):
        raise Exception(f"Ngram argument {n=} must >=1")
    result = list()
    sentence=list()
    sentence = (n-1)*['[START]']
    for token in doc:
        if token.is_alpha:
            sentence.append(token)
    sentence+=['[END]']
    for word in range(len(sentence) - (n-1)):
        words=[]
        for i in range(n):
            try:
                words.append(sentence[word+i].text)
            except AttributeError:
                words.append(sentence[word+i])
        result.append(words)
    return result


"""
Gets a list of ngrams and returns 
a list dictionary with ngrams as keys
and their counts in the corpus as values
"""
def get_ngram_counts(ngrams): 
    ngram_dict = {}
    for stanza in ngrams:
        for ngram in stanza:
            ngram_str=' '.join(map(str,ngram))
            ngram_dict[ngram_str]=ngram_dict[ngram_str]+1 if ngram_str in ngram_dict else 1
    if(np.shape(ngrams[0][0])==(1,)):
        ngram_dict['[START]']=ngram_dict['[END]']
    return ngram_dict


"""
Gets a list of ngrams and returns 
a list dictionary with ngrams as keys
and their counts in the corpus as values
"""
def ngram_prob(ngrams,n_minus1_grams,vocab,n=2):
    ngram_dict={}
    for i in ngrams.keys():
        i_str=''.join(map(str,i))
        prev_ngram_str=' '.join(map(str,i.split()[0:n-1]))
        try: #bigrams
        #Calculating ngram conditional probabiltiy and applying laplace smoothing
            ngram_dict[i_str]=(ngrams[i_str]+1)/(n_minus1_grams[prev_ngram_str]+len(vocab)) 
        except: #trigrams
            prev_ngram_str=' '.join(map(str,i.split()[1:n]))
            ngram_dict[i_str]=(ngrams[i_str]+1)/(n_minus1_grams[prev_ngram_str]+len(vocab)) 
    return ngram_dict


"""
Gets vocabulary size (number of unique words)
of a corpus. Used for laplace smoothing
"""
def get_unique_words(corpus):
    vocab_set = set()
    for verse in corpus:
        words=verse.split()
        for word in words:
            if word not in vocab_set:
                vocab_set.add(word)
            else:
                pass
    return vocab_set


"""
Returns a sorted ngram dictonary(in descending order of their probability),
based on a prior key (e.g if input is "kiya", outputs dictionary of ngrams
containing, but not limited to, "kiya", with their probabilities as well)
"""
def getkey(k,dic):
    dic=dict(sorted(dic.items(), key=lambda x:x[1],reverse=True))
    return (dict([(key,value) for key, value in dic.items() if k in key]))


"""
Returns an ngram with the highest probability 
from an ngram dictionary
"""
def get_highest_prob_word(word,ngrams,c=1):
    high_prob_words=getkey(word,ngrams)
    list_high_prob_words=list(high_prob_words)
    
    curr_word=''
    try:
        curr_word=list_high_prob_words[0]
        while('[END]' in curr_word or '[START]' in curr_word):
            curr_word=list_high_prob_words[c]
            c+=1
    except:
        pass
    return curr_word


"""
Returns an ngram with the highest probability 
from an ngram dictionary, but looks both
forward and backwards ways, and returns 
whichever one has the highest probability
"""
def get_highest_prob_bidirectional(word,ngrams,c=1):
    high_prob_words=getkey(word+" ",ngrams)
    list_high_prob_words=list(high_prob_words)
    
    rev_high_prob_words=getkey(" "+word,ngrams)
    rev_list_high_prob_words=list(rev_high_prob_words)
    
    curr_word=''
    curr_word_prev=''
    try:
        curr_word=list_high_prob_words[0]
        curr_word_prev= rev_list_high_prob_words[0]
        
        if high_prob_words[curr_word]>=rev_high_prob_words[curr_word_prev]:
            chosen_word=curr_word
            flag=0
        else:
            chosen_word=curr_word_prev
            flag=1
        while('[END]' in chosen_word or '[START]' in chosen_word):
            curr_word=list_high_prob_words[c]
            curr_word_prev= rev_list_high_prob_words[c]
            if high_prob_words[curr_word]>=rev_high_prob_words[curr_word_prev]:
                chosen_word=curr_word
                flag=0
            else:
                chosen_word=curr_word_prev
                flag=1
            c+=1
    except:
        pass
    chosen_word=chosen_word.split()[-1] if not flag else chosen_word.split()[0]
    return chosen_word


"""
This is soundex function taken from the Jellyfish
package (https://github.com/jamesturk/jellyfish/blob/main/jellyfish/_jellyfish.py)
All credits to the original authors jamesturk, juliangilbey et al.
This basically returns the rhyming signature of a word, which we 
will later use to predict ending rhyming words for our 'ghazal'
"""
def rhyme_signature(s):
    if not s:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = s.upper()
    replacements = (
        ("BFPV", "1"),
        ("CGJKQSXZ", "2"),
        ("DT", "3"),
        ("L", "4"),
        ("MN", "5"),
        ("R", "6"),
    )
    result = [s[0]]
    count = 1
    #find replacment for first character
    for lset, sub in replacements:
        if s[0] in lset:
            last = sub
            break
    else:
        last = None
    for letter in s[1:]:
        for lset, sub in replacements:
            if letter in lset:
                if sub != last:
                    result.append(sub)
                    count += 1
                last = sub
                break
        else:
            if letter != "H" and letter != "W":
                #leave last alone if middle letter is H or W
                last = None
        if count == 4:
            break
    result += "0" * (4 - count)
    return "".join(result)


"""
Driver function to check whether two 
words rhyme, utilizing our previous
rhyme_signature function
"""
def is_rhyming(w1,w2):
    w1s=rhyme_signature(w1)
    w2s=rhyme_signature(w2)
    
    return True if (w1s[1]==w2s[1] and w1s[2]==w2s[2] and w1s[3]==w2s[3]) else False 

# Main generation functions

In [4]:
"""
Main driver function which generates a verse,
given ngram probability dictionary, ngram length,
and whether to generate standard or backward n-gram model
"""
def gen_verse(ngrams,ngramlength,generated_verse,curr_verse_no,reverse=False):
    if(ngramlength!=2 and ngramlength!=3):
        raise Exception("Only works on bigrams and trigrams (n=2 or n=3)")
    verse=[]
    n=random.randint(6,8)
    for i in range(n):
        if(i==0): #START
            start_word=getkey('[START]',ngrams) if not reverse else getkey('[END]',ngrams)
            verse.append("".join(re.sub("[\(\[].*?[\)\]]", "",random.choices(list(start_word.keys()), weights=start_word.values(), k=1)[0]).split()[-1]))
        elif i==n-1: #END
            end_word=getkey('[END]',ngrams) if not reverse else getkey('[START]',ngrams)
            end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
            if(curr_verse_no%2==0):
                c=0
                #Attempts to create rhyming stanza, as stated in the BONUS task
                while not is_rhyming(generated_verse[curr_verse_no-2].split()[-1],end_word_cleaned):
                    try:
                        end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
                    except:
                        end_word=getkey('[END]',ngrams) if not reverse else getkey('[START]',ngrams)
                        end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
                        break
                    if(c==20):
                        break
            verse.append(end_word_cleaned)
        else:
            #try:
            if ngramlength==2 or i<=2:
                mid_word=get_highest_prob_word(verse[i-1]+" ",ngrams)
            else:
                prec_word=verse[i-1]+" " +verse[i-2]
                mid_word=get_highest_prob_word(prec_word+" ",ngrams)
            try:
                mid_word=mid_word.split()[-1]
            except:
                while((len(mid_word)==0) or ('[START]' in mid_word) or ('[END]' in mid_word)):
                    mid_word=(random.choices(list(ngrams.keys()), weights=ngrams.values(), k=1)[0]).split()[-1]
            counter=0
            while((len(mid_word)==0) or ('[START]' in mid_word) or ('[END]' in mid_word)):
                if(counter>5):
                    mid_word=(random.choices(list(ngrams.keys()), weights=ngrams.values(), k=1)[0]).split()[-1]
                    break
                if ngramlength==2 or i<=2:
                    mid_word=get_highest_prob_word(verse[i-1]+" ",ngrams)
                else:
                    prec_word=verse[i-1]+" " +verse[i-2]
                    mid_word=get_highest_prob_word(prec_word+" ",ngrams)
                counter+=1
            mid_word=mid_word.split()[-1]
            verse.append(mid_word)
    if(reverse):
        verse[0], verse[-1] = verse[-1], verse[0]
    return " ".join(verse) if not reverse else " ".join([i for i in verse[::-1]])


"""
Main driver function which generates a verse,
using bidirectional bigram model
"""
def bidirectional_bigram(ngrams,ngramlength,generated_verse,curr_verse_no):
    if(ngramlength!=2):
        raise Exception("Only works on bigrams and trigrams (n=2 or n=3)")
    verse=[]
    n=random.randint(6,8)
    for i in range(n):
        if(i==0): #START
            start_word=getkey('[START]',ngrams)
            verse.append("".join(re.sub("[\(\[].*?[\)\]]", "",random.choices(list(start_word.keys()), weights=start_word.values(), k=1)[0]).split()[-1]))
        elif i==n-1: #END
            end_word=getkey('[END]',ngrams)
            end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
            if(curr_verse_no%2==0):
                c=0
                #Attempts to create rhyming stanza, as stated in the BONUS task
                while not is_rhyming(generated_verse[curr_verse_no-2].split()[-1],end_word_cleaned):
                    try:
                        end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
                    except:
                        end_word=getkey('[END]',ngrams) if not reverse else getkey('[START]',ngrams)
                        end_word_cleaned="".join(re.sub("[\(\[].*?[\)\]]", "", random.choices(list(end_word.keys()), weights=end_word.values(), k=1)[0]).split()[-1])
                    if(c==20):
                        break
            verse.append(end_word_cleaned)
        else:
            mid_word=get_highest_prob_bidirectional(verse[i-1],ngrams)
            counter=0
            while((len(mid_word)==0) or ('[START]' in mid_word) or ('[END]' in mid_word)):
                if(counter>5):
                    mid_word=(random.choices(list(ngrams.keys()), weights=ngrams.values(), k=1)[0]).split()[-1]
                    break
                mid_word=get_highest_prob_bidirectional(verse[i-1],ngrams)
                counter+=1
                mid_word=mid_word.split()[-1]
            verse.append(mid_word)    
    return " ".join(verse)

<br><br>
# Reading corpus and building ngrams

In [5]:
poetry=read("poetry_english.txt")
unique_vocab=get_unique_words(poetry)

unigrams=[]
bigrams=[]
trigrams=[]
for i in poetry:
    unigrams.append(get_ngrams(nlp(i),1))
    bigrams.append(get_ngrams(nlp(i)))
    trigrams.append(get_ngrams(nlp(i),3))


unigrams_count=get_ngram_counts(unigrams)
bigrams_count=get_ngram_counts(bigrams)
trigrams_count=get_ngram_counts(trigrams)

bigrams_prob=ngram_prob(bigrams_count,unigrams_count,unique_vocab,2)
trigrams_prob=ngram_prob(trigrams_count,bigrams_count,unique_vocab,3)

In [6]:
stanzas=7
verse=2

# Standard n-gram Models

### Bigram

In [7]:
forward_bigram_ghazal=[]
for i in range(1,(stanzas*verse)+1):
    forward_bigram_ghazal.append(upper_case.sub(capital,(gen_verse(bigrams_prob,2,forward_bigram_ghazal,i))))    
for idx,i in enumerate(forward_bigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Dhamki mein bhi nahi aati hai ke gaye
Yeh baat par nahi aati hai tha


Mohammad bhi nahi aati hai ke liye Aziz
Ik baar hota hai ke liye hum bakhash


Mohammad bhi nahi aati hai ghalib
Aayi ke liye hum ne chaha tha ghalib


Beddad dost naaseh ne chaha tha ke bani
Ghuncha phir laga ke liye hum mein


Nah sun hwa hai ke liye dalie
Rakh kar raha hai ke liye hum qaail


Pucho to kya hai ke hoga
Shairi unki adab hon mein bhi nahi likhwaye


Yahan ke liye hum ne ghalib
Taaqat guftaar aur bhi nahi ghalib




### Trigram

In [8]:
forward_trigram_ghazal=[]
for i in range(1,(stanzas*verse)+1):
    forward_trigram_ghazal.append(upper_case.sub(capital,(gen_verse(trigrams_prob,3,forward_trigram_ghazal,i)))) 
for idx,i in enumerate(forward_trigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Aam bhi hain thi mein visale
Baat par hwa o ishhq nikla


Moay deedaa dil hui mujh par aaya
Subha Marghoob buut siya to kya


Bas gashta wafa kidhar hain tairay ho
Zuba ki dawa bah dushman ke ko se


Parhnay ke liya ke saamna apne hai
Barg Gul gird wagerna Daagh nahi


Hi nah hwa maang kya khula
Wahn aarai ko to koi ko ka qaail


Sheree sukhan se khaak pouchon mujhe
Rasam barham fanaa se ghaafil saaz


Koi dil jigar rawish jis nahi dosh Rubab
Beddad dost naaseh lagey jo barq bahar shabab




<br><br>

# Backward n-gram Models

### Backward Bigram

In [9]:
backward_bigram_ghazal=[]
for i in range(1,(stanzas*verse)+1):
    backward_bigram_ghazal.append(upper_case.sub(capital,(gen_verse(bigrams_prob,2,backward_bigram_ghazal,i,reverse=True))))   
for idx,i in enumerate(backward_bigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Gayi hum liye ke hai juz
Hai tha chaha ne hum liye ke juz


Se nahi bhi mein guzar tre
Hwa chaha ne hum liye ke hai tri


Doori ke hai aati nahi bhi mein ik
Chahiye ke hai hota baar ik ik


Ghalib nayam aish terhan ki duniya
Gaya ke hai kya to pion


Bas hum liye ke hai khoo go
Haal tha chaha ne hum liye e to


Hum hum liye ke tha chaha ne bharam
Paaya hai aati nahi bhi mein ko Mehram


Nikla nahi bhi woh so marey
Sunaoun hum liye e hasrat tri




### Backward Trigram

In [10]:
backward_trigram_ghazal=[]
for i in range(1,(stanzas*verse)+1):
    backward_trigram_ghazal.append(upper_case.sub(capital,(gen_verse(trigrams_prob,3,backward_trigram_ghazal,i,reverse=True))))   
for idx,i in enumerate(backward_trigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Sharminda kareem qayamat tha nah maienay aati
Sharminda se dar kehte nah maienay deta


Par ghareeb yeh dil hai hwa karta
Tak mudda hi jo hai hai baqi parda


Umeed ke kar ne se ne
Yaktaa Gul ke masti nah ulajh ke


Tha zarraa bhar bhar liya ke hai
Roshni izr bhi o Danish taa


Sahi mansoob o mann raha hain bhi zulm
Maloom mahaal par usay dikhavay ke is ilm


Doori he ho ki assar be ki
Tha kefiyat aur dekh same liya ke woh


Nahi kaaba tha o nahi mein dardi
Kya sahi hai to hota dard




<br><br>
# Bidirectional Bigram

In [11]:
bidirectional_bigram_ghazal=[]
for i in range(1,(stanzas*verse)+1):
    bidirectional_bigram_ghazal.append(upper_case.sub(capital,(bidirectional_bigram(bigrams_prob,2,bidirectional_bigram_ghazal,i))))    
for idx,i in enumerate(bidirectional_bigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Warna yaan warna yaan warna yaan pewand
Mein hon mein hon mein hon wandi


Ishhq ko uss ko uss ko lolaak
Dozakh ko uss ko uss ko talak


Mein hon mein hon mein hon ko
Kyun nah sun hwa sun hwa sun liye


Hmm sath piyay sath piyay diya
Qanaat nah sun hwa sun hai


Nah sun hwa sun hwa sun hwa kshad
Nah sun hwa sun hwa sayyad


Khoo hai ke liye ke hoga
Fiqiya shehar mein hon mein hon jisay


Azad ajab azad ajab azad ajab azad shama
Kharaaj diyaa viran diyaa viran hain




<br> <br>
### Comparing output of Bidirectional Bigram Model and Trigram Model

#### Trigram

In [12]:
for idx,i in enumerate(forward_trigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Aam bhi hain thi mein visale
Baat par hwa o ishhq nikla


Moay deedaa dil hui mujh par aaya
Subha Marghoob buut siya to kya


Bas gashta wafa kidhar hain tairay ho
Zuba ki dawa bah dushman ke ko se


Parhnay ke liya ke saamna apne hai
Barg Gul gird wagerna Daagh nahi


Hi nah hwa maang kya khula
Wahn aarai ko to koi ko ka qaail


Sheree sukhan se khaak pouchon mujhe
Rasam barham fanaa se ghaafil saaz


Koi dil jigar rawish jis nahi dosh Rubab
Beddad dost naaseh lagey jo barq bahar shabab




#### Bidirectional Bigram Model

In [13]:
for idx,i in enumerate(bidirectional_bigram_ghazal):
    print(i)
    if(idx%2):
        print("\n")

Warna yaan warna yaan warna yaan pewand
Mein hon mein hon mein hon wandi


Ishhq ko uss ko uss ko lolaak
Dozakh ko uss ko uss ko talak


Mein hon mein hon mein hon ko
Kyun nah sun hwa sun hwa sun liye


Hmm sath piyay sath piyay diya
Qanaat nah sun hwa sun hai


Nah sun hwa sun hwa sun hwa kshad
Nah sun hwa sun hwa sayyad


Khoo hai ke liye ke hoga
Fiqiya shehar mein hon mein hon jisay


Azad ajab azad ajab azad ajab azad shama
Kharaaj diyaa viran diyaa viran hain




#### Both Bidirectional Bigram and Trigram Models produce a similar output, acceptable output. Both are also attempting to preserve the rhyming structure of the stanzas that I implemented